> Read VAE and VQ-VAE before coming to this blog. 

> We will discuss about VQVAE and how autoregressive image modelling works.

> We will also discuss how conditioning is applied to these models. 


VQ-GAN is a family of models which uses auto encoding to reduce the dimensionality of the image and use transformers for the generative aspect of it. 

The model is trained in two stages.
- Train Encoder, quantizer and decoder.
- taking quantizer outputs to train a generative model using autoregressive transformers. 


> The paper also talks about generating megapixel images by using a sling window approch conditioning on previous windows. 

> As usual we will try to make this notebook as informative as possible with minimum dependencies.

## VQGAN
the paper claims that the quality of the reconstructions using VAE detoriates as we increase the receptive field. so we will use a GAN to train the E-Q-D architecture. The model uses LPIPS loss instead of L2 loss, also adds a patch based discrimintor along with commitement and codebook loss (discussed in vqvae).

In [1]:
import torch
import os 
import torchvision
import numpy as np 
from tqdm import tqdm
from diffusers import VQModel

/Users/prakashjay/Downloads/genai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load MNIST data and resize it to 64x64
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('data/', train=True, download=True,
                                transform=torchvision.transforms.Compose([
                                    torchvision.transforms.Pad(2),
                                    torchvision.transforms.ToTensor()
                                ])),
    batch_size=64, shuffle=True)

val_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('data/', train=False, download=True,
                                transform=torchvision.transforms.Compose([
                                    torchvision.transforms.Pad(2),
                                    torchvision.transforms.ToTensor()
                                ])),
    batch_size=64, shuffle=False)

len(train_loader), len(val_loader)

(938, 157)

In [3]:
imgs, labels = next(iter(val_loader))
imgs.shape, labels.shape

(torch.Size([64, 1, 32, 32]), torch.Size([64]))

In [4]:
torchvision.transforms.ToPILImage()(torchvision.utils.make_grid(imgs, nrow=8))

> we will use the VQModel class from the diffusers library to define the VQGAN model.

> we will train for 30 epochs with very small learning rate. 

In [5]:
# embed_dim: 256
# n_embed: 8192 
model = VQModel(
    in_channels=1,
    out_channels=1,
    down_block_types=('DownEncoderBlock2D', 'DownEncoderBlock2D', 'AttnDownEncoderBlock2D'),
    up_block_types=('UpDecoderBlock2D', 'UpDecoderBlock2D', 'AttnUpDecoderBlock2D'),
    block_out_channels=(32, 64, 128),
    layers_per_block=1,
    act_fn='silu',
    latent_channels=4,
    sample_size=32,
    num_vq_embeddings=1024,
    norm_num_groups=32,
    vq_embed_dim=None,
)

In [6]:
params = 0
for param in model.parameters():
    params += param.numel()
params


2775245

## GAN 
In GANs, we have a generator and a discriminator. The discriminator needs to distinguish between the real and fake samples. 

We update the discriminator weights using binary cross entropy loss where real samples should be classified as 1 and fake samples as 0. The generator (encoder and decoder) is updated using a generator loss obtained by trying to fool the discriminator into classifying generated images as real (1).

Discriminator loss: $$ \mathcal{L}_D = -\mathbb{E}_{x \sim p_{data}}[\log D(x)] - \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))] $$

Generator loss: $$ \mathcal{L}_G = -\mathbb{E}_{z \sim p_z}[\log D(G(z))] $$


The equations show the standard GAN loss functions where:
- The discriminator tries to maximize the probability of correctly classifying both real and fake samples
- The generator tries to minimize the probability of the discriminator correctly identifying its outputs as fake

> The following things needs to be implemented to train a VQGAN
- discrimintor 
- GAN loss 
- lpips 
- commitment and codebook loss 
- $\lambda$ - adaptive weight. 


## Final losses
- the generator loss is defined as  genloss + adaptive_weight * (rec loss + lpips_loss + commitment loss + code book loss)
- the discriminator loss is just the binary cross entropy lass as seen above. 


## Discriminator 
In the original paper they implemented Patch based discrimintor obtained from pix2pix paper. we will implement a variant of it here. the original paper tried to generate mega pixel size images and hence patched inputs before sending it to the network. Here we don't need it as our images are already very small in size.

In [7]:
def _conv(input_nc, ndf, ks, stride, pad, norm=True):
    conv = torch.nn.Conv2d(input_nc, ndf, kernel_size=ks, stride=stride, padding=pad)
    act = torch.nn.LeakyReLU(0.2, True)
    out = [conv, act]
    if norm: out.append(torch.nn.BatchNorm2d(ndf))
    return torch.nn.Sequential(*out)

class Disc(torch.nn.Module):
    def __init__(self, input_nc=3, ndf=64):
        super(Disc, self).__init__()
        self.layer1 = _conv(input_nc, ndf, 4, 2, 1, norm=False)
        self.layer2 = _conv(ndf, ndf*2, 4, 2, 1)
        self.layer3 = _conv(ndf*2, ndf*2, 4, 2, 1)
        self.layer4 = _conv(ndf*2, ndf*4, 4, 1, 1)
        self.final = torch.nn.Conv2d(ndf*4, 1, kernel_size=4, padding=1)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.final(x)
        return x 

In [8]:
disc = Disc(1, 32)
disc(imgs).shape

torch.Size([64, 1, 2, 2])

In [9]:
params = 0
for param in disc.parameters():
    params+=param.numel()
params

232737

## LPIPS and GAN loss implementation 

> for LPIPS we will use [lpips package](https://github.com/richzhang/PerceptualSimilarity/tree/master) directly. 

> we will use MSELoss as defined in LSGAN paper.

In [10]:
import lpips
class PerceptionLoss(torch.nn.Module):
    def __init__(self):
        super(PerceptionLoss, self).__init__()
        self.lpips = lpips.LPIPS(net="vgg")

    def forward(self, generated, target):
        return self.lpips(generated, target).mean()


class GANLoss(torch.nn.Module):
    """
    GAN loss
    """

    def __init__(self, use_lsgan=True, target_real_label=1.0, target_fake_label=0.0):
        super(GANLoss, self).__init__()
        self.register_buffer("real_label", torch.tensor(target_real_label))
        self.register_buffer("fake_label", torch.tensor(target_fake_label))
        if use_lsgan:
            self.loss = torch.nn.MSELoss()
        else:
            self.loss = torch.nn.BCELoss()

    def get_target_tensor(self, input, target_is_real):
        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label
        return target_tensor.expand_as(input)

    def forward(self, input, target_is_real):
        target_tensor = self.get_target_tensor(input, target_is_real)
        return self.loss(input, target_tensor)
    

    def generator_loss(self, generated):
        return self.forward(generated, True)

    def discriminator_loss(self, generated, real):
        real_loss = self.forward(real, True)
        fake_loss = self.forward(generated, False)
        return (real_loss + fake_loss) / 2
    
    

## VQGAN 
> the model takes in an image and returns the decoded image, loss, min encodings and min encoding indices. 

In [11]:
class VQGAN(torch.nn.Module):
    def __init__(self, vqmodel):
        super(VQGAN, self).__init__()
        self.model = vqmodel
    
    def forward(self, x):
        z_q, loss, perpexlity, min_encodings, min_encoding_indices = self.encode(x)
        post_quant_conv = self.model.post_quant_conv(z_q)
        decoder = self.model.decode(post_quant_conv)
        return decoder.sample, loss, perpexlity, min_encodings, min_encoding_indices
    

    def encode(self, x):
        encoder = self.model.encode(x).latents
        quant_conv = self.model.quant_conv(encoder)
        z_q, loss, (perpexlity, min_encodings, min_encoding_indices) = self.model.quantize(quant_conv)
        return z_q, loss, perpexlity, min_encodings, min_encoding_indices
    

    def decode(self, quant):
        post_quant_conv = self.model.post_quant_conv(quant)
        decoder = self.model.decode(post_quant_conv)
        return decoder 

In [12]:
encoder = model.encode(imgs).latents
quant_conv = model.quant_conv(encoder)
quant_conv.shape

torch.Size([64, 4, 8, 8])

In [13]:
vqgan = VQGAN(model)
vqgan.to(device='mps')
z_q, loss, perpexlity, min_encodings, min_encoding_indices = vqgan(imgs.to("mps"))
z_q.shape, loss, perpexlity

(torch.Size([64, 1, 32, 32]),
 tensor(0.0495, device='mps:0', grad_fn=<AddBackward0>),
 None)

## Training code

In [14]:
# I usually clip the graidents while training GAN models to make them stable. 
def clip_gradients(model, clip_value):
    for param in model.parameters():
        if param.grad is not None:
            torch.nn.utils.clip_grad_norm_(param, clip_value)

def check_gradient_magnitudes(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm**0.5
    return total_norm

In [15]:
epochs = 30 
disc = Disc(1, 32).to("mps")
opt_d = torch.optim.AdamW(disc.parameters(), lr=0.001)
opt_g = torch.optim.AdamW(vqgan.parameters(), lr=0.001)
gan_loss = GANLoss().to("mps")
perception_loss = PerceptionLoss().to("mps")

iter=0
train_loss = []
val_loss = []
for epoch in range(epochs):
    #train
    disc.train()
    vqgan.train()
    with tqdm(train_loader, total=len(train_loader), desc="Training", leave=False) as pbar:
        for i, (x, _) in enumerate(pbar):

            x = x.to(device='mps')
            x_out, qloss, perpexlity, min_encodings, min_encoding_indices = vqgan(x)

            # update discrimintor
            opt_d.zero_grad()
            fake = disc(x_out.detach())
            real = disc(x)
            dloss = gan_loss.discriminator_loss(fake, real)
            dloss.backward()
            clip_gradients(disc, 0.05)
            opt_d.step()


            #update generator 
            opt_g.zero_grad()
            fake = disc(x_out)
            ploss = perception_loss(x_out, x)
            gloss = gan_loss.generator_loss(fake)
            total_g_loss = gloss+ploss+qloss 
            total_g_loss.backward()
            clip_gradients(vqgan, 0.1)
            opt_g.step()
            iter+=1
            pbar.set_postfix(gloss=gloss.item(), dloss=dloss.item(), qloss=qloss.item(), ploss=ploss.item())
            train_loss.append([gloss.item(), dloss.item(), qloss.item(), ploss.item()])

    #val
    vqgan.eval()
    disc.eval()
    with tqdm(val_loader, total=len(val_loader), desc="Validating", leave=False) as pbar:
        for i, (x, _) in enumerate(pbar):
            x = x.to(device='mps')
            x_out, qloss, perpexlity, min_encodings, min_encoding_indices = vqgan(x)
            dloss = gan_loss.discriminator_loss(disc(x_out.detach()), disc(x))
            fake = disc(x_out)
            gloss = gan_loss.generator_loss(fake)
            ploss = perception_loss(x_out, x)
            val_loss.append([gloss.item(), dloss.item(), qloss.item(), ploss.item()])
            pbar.set_postfix(gloss=gloss.item(), dloss=dloss.item(), qloss=qloss.item(), ploss=ploss.item())
            if i == 0:
                gen = x_out.detach().cpu()

    #Generate some images 
    #Generate some images
    generated_imgs = torchvision.transforms.ToPILImage()(torchvision.utils.make_grid(gen, nrow=8))
    if not os.path.exists('generated'):
        os.makedirs('generated')
    generated_imgs.save(f'generated/vqgan_samples_{epoch+1}.png')
    

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/Users/prakashjay/Downloads/genai/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/prakashjay/Downloads/genai/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/prakashjay/Downloads/genai/.venv/lib/python3.10/site-packages/lpips/weights/v0.1/vgg.pth


KeyboardInterrupt: 